In [31]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import numpy as np
import datetime
import data_loader
import plot_tools
import rbsp_chorus_tool
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import spacepy.time

import matplotlib.pyplot as plt
from matplotlib import colors

import importlib
importlib.reload(data_loader)
importlib.reload(plot_tools)
importlib.reload(rbsp_chorus_tool)



%matplotlib qt

In [32]:
#Parameters
#Dates
start = datetime.datetime(year = 2013, month = 1, day = 1)
end = datetime.datetime(year = 2013, month = 5, day = 1)
epoch_bin_size = datetime.timedelta(hours = 3)

#L
L_bin_size = 0.10
L_start = 3.5
L_end = 7.0

#MLT
mlt_start = 0
mlt_end = 24

In [3]:
satellite_names = ["metop01", "metop02", "metop03", "noaa15", "noaa16", "noaa17", "noaa18", "noaa19"]

satellites = []

for sat in satellite_names:
    
    data = data_loader.load_compressed_poes_data(satellite = sat,
                                                 start = start,
                                                 end = end)
            
    if len(data["EPOCH"]) == 0:
        continue
    
    satellites.append(data)


Loading : POES_2013_metop01_CLEAN.npz
Loading : POES_2013_metop02_CLEAN.npz
Unable to find any compressed POES data for: metop03

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\metop03\POES_2013_metop03_CLEAN.npz, continuing...!
Loading : POES_2013_noaa15_CLEAN.npz
Loading : POES_2013_noaa16_CLEAN.npz
Loading : POES_2013_noaa17_CLEAN.npz
Loading : POES_2013_noaa18_CLEAN.npz
Loading : POES_2013_noaa19_CLEAN.npz


In [33]:
P = 1
epoch_POES = np.zeros(shape=(0))
L_POES = np.zeros(shape=(0))
mlt_POES = np.zeros(shape=(0))
naive_chorus_POES = np.zeros(shape=(0))

for i, sat in enumerate(satellites):
    
    print(f"Processing satellite: {satellite_names[i]}")
    
    #Calculate Chorus
    j_40 = sat["MEP_ELE_TEL0_FLUX_E1"]
    j_130 = sat["MEP_ELE_TEL0_FLUX_E2"]

    J = j_40 - j_130
    # For some stupid reason this subtraction rarely creates negatives even though the channels are integral... :(
    J[J < 0] = np.NaN

    naive_chorus_intensity = J / (P * ((sat["L"] - 3)**2 + 0.03))
    naive_chorus_amplitudes = np.sqrt(naive_chorus_intensity)
      
    #Concatenate all the data
    epoch_POES = np.concatenate((epoch_POES, sat["EPOCH"]), axis = 0)
    L_POES = np.concatenate((L_POES, sat["L"]), axis = 0)
    mlt_POES = np.concatenate((mlt_POES, sat["MLT"]), axis = 0)
    naive_chorus_POES = np.concatenate((naive_chorus_POES, naive_chorus_amplitudes), axis = 0)

within_mlt_range_POES = (mlt_start <= mlt_POES) & (mlt_POES < mlt_end)
within_L_range_POES = (L_start < L_POES) & (L_POES < L_end)
within_epoch_range_POES = (start < epoch_POES) & (epoch_POES < end)
within_all_ranges_POES = within_mlt_range_POES & within_L_range_POES & within_epoch_range_POES
finite_chorus_POES = np.isfinite(naive_chorus_POES)

epoch_POES = epoch_POES[within_all_ranges_POES & finite_chorus_POES]
L_POES = L_POES[within_all_ranges_POES & finite_chorus_POES]
mlt_POES = mlt_POES[within_all_ranges_POES & finite_chorus_POES]
naive_chorus_POES = naive_chorus_POES[within_all_ranges_POES & finite_chorus_POES]

cumulative_chorus_POES, num_points_in_each_epoch_L_bin_POES = plot_tools.bin_3D_data(xdata = epoch_POES,
                                                                                     ydata = L_POES, 
                                                                                     zdata = naive_chorus_POES,
                                                                                     xstart = start,
                                                                                     xend = end,
                                                                                     xstep = epoch_bin_size,
                                                                                     ystart = L_start,
                                                                                     yend = L_end,
                                                                                     ystep = L_bin_size)


average_chorus_POES = cumulative_chorus_POES / num_points_in_each_epoch_L_bin_POES


Processing satellite: metop01
Processing satellite: metop02
Processing satellite: metop03
Processing satellite: noaa15
Processing satellite: noaa16
Processing satellite: noaa17
Processing satellite: noaa18


C:\Users\brofe\AppData\Local\Temp\ipykernel_14172\2283224205.py:50: RuntimeWarning: invalid value encountered in divide
  average_chorus_POES = cumulative_chorus_POES / num_points_in_each_epoch_L_bin_POES


In [34]:
average_chorus_POES = average_chorus_POES + 1e-31 #Stabilize the log norm

fig, ax = plt.subplots(1, 1, figsize=(16, 9), sharex=True)

image = ax.imshow(average_chorus_POES.T, 
                  origin = "lower",
                  extent=[start, end, L_start, L_end],
                  norm=colors.LogNorm(vmin=1e-1, vmax=1000),
                  aspect="auto",
                  interpolation="none")

image.cmap.set_under("black")

cbar = plt.colorbar(image, ax=ax, pad=0.01)

cbar.set_label("Chorus Bw (pT)\n", loc="center", labelpad=15, rotation=270)

ax.set_title("POES Proxy Chorus")
ax.set_ylabel("L")
ax.set_xlabel("Time")

plt.tight_layout()

In [35]:
WNA_survey_a = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L4", "WNA_SURVEY"],
                                                     start=start,
                                                     end=end,
                                                     satellite="a")

WNA_survey_b = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L4", "WNA_SURVEY"],
                                                     start=start,
                                                     end=end,
                                                     satellite="b")

WFR_spectral_matrix_a = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L2", "WFR_SPECTRAL_MATRIX_DIAGONAL"],
                                                              start=start,
                                                              end=end,
                                                              satellite="a")

WFR_spectral_matrix_b = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L2", "WFR_SPECTRAL_MATRIX_DIAGONAL"],
                                                              start=start,
                                                              end=end,
                                                              satellite="b")

In [36]:
mlt_A = WNA_survey_a["MLT"]
L_A = WNA_survey_a["L"]
Epoch_A =  cdfepoch.to_datetime(WNA_survey_a["Epoch"])

chorus_A = rbsp_chorus_tool.iterate_through_days_and_calculate_chorus_amplitudes(WNA_survey = WNA_survey_a,
                                                                                 WFR_spectral_matrix = WFR_spectral_matrix_a)


within_mlt_range_A = (mlt_start <= mlt_A) & (mlt_A < mlt_end)
within_L_range_A = (L_start < L_A) & (L_A < L_end)
within_epoch_range_A = (start < Epoch_A) & (Epoch_A < end)
within_all_ranges_A = within_mlt_range_A & within_L_range_A & within_epoch_range_A
finite_chorus_A = np.isfinite(chorus_A)

Epoch_A = Epoch_A[within_all_ranges_A & finite_chorus_A]
L_A = L_A[within_all_ranges_A & finite_chorus_A]
mlt_A = mlt_A[within_all_ranges_A & finite_chorus_A]
chorus_A = chorus_A[within_all_ranges_A & finite_chorus_A]


mlt_B = WNA_survey_b["MLT"]
L_B = WNA_survey_b["L"]
Epoch_B = cdfepoch.to_datetime(WNA_survey_b["Epoch"])

chorus_B = rbsp_chorus_tool.iterate_through_days_and_calculate_chorus_amplitudes(WNA_survey = WNA_survey_b,
                                                                                 WFR_spectral_matrix = WFR_spectral_matrix_b)

within_mlt_range_B = (mlt_start <= mlt_B) & (mlt_B < mlt_end)
within_L_range_B = (L_start < L_B) & (L_B < L_end)
within_epoch_range_B = (start < Epoch_B) & (Epoch_B < end)
within_all_ranges_B = within_mlt_range_B & within_L_range_B & within_epoch_range_B
finite_chorus_B = np.isfinite(chorus_B)

Epoch_B = Epoch_B[within_all_ranges_B & finite_chorus_B]
L_B = L_B[within_all_ranges_B & finite_chorus_B]
mlt_B = mlt_B[within_all_ranges_B & finite_chorus_B]
chorus_B = chorus_B[within_all_ranges_B & finite_chorus_B]

print(Epoch_A.shape)
print(chorus_A.shape)
print(L_A.shape)

print(Epoch_B.shape)
print(chorus_B.shape)
print(L_B.shape)

(1247417,)
(1247417,)
(1247417,)
(1251144,)
(1251144,)
(1251144,)


In [37]:
cumulative_chorus_A, num_points_in_each_epoch_L_bin_A = plot_tools.bin_3D_data(xdata = Epoch_A,
                                                                               ydata = L_A, 
                                                                               zdata = chorus_A,
                                                                               xstart = start,
                                                                               xend = end,
                                                                               xstep = epoch_bin_size,
                                                                               ystart = L_start,
                                                                               yend = L_end,
                                                                               ystep = L_bin_size)


average_chorus_A = cumulative_chorus_A / num_points_in_each_epoch_L_bin_A


cumulative_chorus_B, num_points_in_each_epoch_L_bin_B = plot_tools.bin_3D_data(xdata = Epoch_B,
                                                                               ydata = L_B, 
                                                                               zdata = chorus_B,
                                                                               xstart = start,
                                                                               xend = end,
                                                                               xstep = epoch_bin_size,
                                                                               ystart = L_start,
                                                                               yend = L_end,
                                                                               ystep = L_bin_size)


average_chorus_B = cumulative_chorus_B / num_points_in_each_epoch_L_bin_B

average_chorus_RBSP = np.nansum([average_chorus_A, average_chorus_B], axis=0) / 2.0

C:\Users\brofe\AppData\Local\Temp\ipykernel_14172\1204160032.py:12: RuntimeWarning: invalid value encountered in divide
  average_chorus_A = cumulative_chorus_A / num_points_in_each_epoch_L_bin_A
C:\Users\brofe\AppData\Local\Temp\ipykernel_14172\1204160032.py:26: RuntimeWarning: invalid value encountered in divide
  average_chorus_B = cumulative_chorus_B / num_points_in_each_epoch_L_bin_B


In [38]:
average_chorus_RBSP[average_chorus_RBSP == 0] = np.nan

fig, ax = plt.subplots(1, 1, figsize=(16, 9), sharex=True)

image = ax.imshow(average_chorus_RBSP.T, 
                  origin = "lower",
                  extent=[start, end, L_start, L_end],
                  norm=colors.LogNorm(vmin=1e-1, vmax=1000),
                  aspect="auto",
                  interpolation="none")

image.cmap.set_under("black")

cbar = plt.colorbar(image, ax=ax, pad=0.01)

cbar.set_label("Chorus Bw (pT)\n", loc="center", labelpad=15, rotation=270)

ax.set_title("RBSP Observed Chorus")
ax.set_ylabel("L")
ax.set_xlabel("Time")

plt.tight_layout()

In [39]:
#Apply correction to POES data to fit RBSP

correction_factors = []

for T in range(average_chorus_RBSP.shape[0]):
    
    POES_naive_chorus = average_chorus_POES[T, :]
    RBSP_chorus = average_chorus_RBSP[T, :]
    
    correction_factors.append(np.nanmean((POES_naive_chorus / RBSP_chorus)))
        
correction_factors = np.expand_dims(np.asarray(correction_factors), axis=1)

corrected_chorus_proxy = average_chorus_POES / correction_factors

C:\Users\brofe\AppData\Local\Temp\ipykernel_14172\1135825255.py:10: RuntimeWarning: Mean of empty slice
  correction_factors.append(np.nanmean((POES_naive_chorus / RBSP_chorus)))


In [40]:
corrected_chorus_proxy = corrected_chorus_proxy + 1e-31 #Stabilize the log norm

fig, ax = plt.subplots(1, 1, figsize=(16, 9), sharex=True)

image = ax.imshow(corrected_chorus_proxy.T, 
                  origin = "lower",
                  extent=[start, end, L_start, L_end],
                  norm=colors.LogNorm(vmin=1e-1, vmax=1000),
                  aspect="auto",
                  interpolation="none")

image.cmap.set_under("black")

cbar = plt.colorbar(image, ax=ax, pad=0.01)

cbar.set_label("Chorus Bw (pT)\n", loc="center", labelpad=15, rotation=270)

ax.set_title("Optimal POES Proxy Chorus")
ax.set_ylabel("L")
ax.set_xlabel("Time")

plt.tight_layout()

In [41]:
quantified_error = np.abs((average_chorus_RBSP - corrected_chorus_proxy) / average_chorus_RBSP) * 100.0

In [42]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


fig, axs = plt.subplots(5, 1, sharex=True)

chorus_plots = [average_chorus_POES, corrected_chorus_proxy, average_chorus_RBSP]
labels = ["Naive POES Proxy (P=1)", "Corrected POES Proxy (Variable P)", "Observed Chorus RBSP"]

for i, plot in enumerate(chorus_plots):
    
    image = axs[i].imshow(plot.T, 
                          origin = "lower",
                          extent=[start, end, L_start, L_end],
                          norm=colors.LogNorm(vmin=1, vmax=100),
                          aspect="auto",
                          interpolation="none")

    image.cmap.set_under("black")

    #To be honest I have no idea how this actually works anymore. Matplotlib is bad and they make it difficult to position the colorbars.
    axins = inset_axes(
        axs[i],
        width="1%",  # width: 5% of parent_bbox width
        height="100%",  # height: 50%
        loc="lower left",
        bbox_to_anchor=(1.01, 0, 1, 1),
        bbox_transform=axs[i].transAxes,
        borderpad=0
    )

    cbar = plt.colorbar(image, cax=axins) 
    cbar.set_label("Chorus Amplitude (pT)\n", loc="center", labelpad=15, rotation=270)

    axs[i].set_title(labels[i])
    axs[i].set_ylabel("L")
    if i == len(chorus_plots) - 1:
        axs[i].set_xlabel("Time")


dates = spacepy.time.tickrange(start = start, end = end - epoch_bin_size, deltadays = epoch_bin_size, dtype="UTC")

axs[len(chorus_plots)].semilogy(dates.UTC, np.squeeze(correction_factors))
axs[len(chorus_plots)].set_ylabel("P")

image = axs[len(chorus_plots) + 1].imshow(quantified_error.T, 
                                          origin = "lower",
                                          extent=[start, end, L_start, L_end],
                                          norm=colors.LogNorm(vmin=1, vmax=1000),
                                          aspect="auto",
                                          interpolation="none")

image.cmap.set_under("black")

#To be honest I have no idea how this actually works anymore. Matplotlib is bad and they make it difficult to position the colorbars.
axins = inset_axes(
    axs[len(chorus_plots) + 1],
    width="1%",  # width: 5% of parent_bbox width
    height="100%",  # height: 50%
    loc="lower left",
    bbox_to_anchor=(1.01, 0, 1, 1),
    bbox_transform=axs[len(chorus_plots) + 1].transAxes,
    borderpad=0
)

cbar = plt.colorbar(image, cax=axins) 
cbar.set_label("Percent Error\n", loc="center", labelpad=15, rotation=270)

In [14]:
OMNI_ONE_MIN = data_loader.load_raw_data_from_config(id=["OMNI", "ONE_MIN_RESOLUTION"], 
                                                      start=start,
                                                      end=end)
OMNI_ONE_HOUR = data_loader.load_raw_data_from_config(id=["OMNI", "ONE_HOUR_RESOLUTION"], 
                                                      start=start,
                                                      end=end)

In [44]:
fig, axs = plt.subplots(3, 1, sharex=True)

dates = spacepy.time.tickrange(start = start, end = end - epoch_bin_size, deltadays = epoch_bin_size, dtype="UTC")

axs[0].semilogy(dates.UTC, np.squeeze(correction_factors))
axs[0].set_ylabel("P")
axs[0].set_title("Optimal P Value")

OMNI_dates = cdfepoch.to_datetime(OMNI_ONE_MIN["Epoch"])

within_time_region = (start <= OMNI_dates) & (OMNI_dates < end)

flow_speed = OMNI_ONE_MIN["flow_speed"]
flow_speed[flow_speed > 90000] = np.nan

flow_speed = flow_speed[within_time_region]
OMNI_dates = OMNI_dates[within_time_region]

axs[1].plot(OMNI_dates, flow_speed)
axs[1].set_ylabel("Flow Speed (km/s)")
axs[1].set_title("Flow Speed")

OMNI_dates_unx = np.asarray([OMNI_dates[i].timestamp() for i in range(len(OMNI_dates))])
P_dates_unx = dates.UNX

not_nan = np.isfinite(flow_speed)
flow_speed_interpolated = np.interp(P_dates_unx, OMNI_dates_unx[not_nan], flow_speed[not_nan], left = np.nan, right = np.nan)


OMNI_HOUR_DATES = cdfepoch.to_datetime(OMNI_ONE_HOUR["Epoch"])

within_time_region = (start <= OMNI_HOUR_DATES) & (OMNI_HOUR_DATES < end)

axs[2].plot(OMNI_HOUR_DATES[within_time_region], OMNI_ONE_HOUR["KP"][within_time_region])
axs[2].set_title("OMNI KP-Index")
axs[2].set_ylabel("KP")

Text(0, 0.5, 'KP')

In [16]:
import scipy.stats

P_vals = np.log(np.squeeze(correction_factors))

not_nan = np.isfinite(P_vals) & np.isfinite(flow_speed_interpolated)

coefficient = scipy.stats.pearsonr(flow_speed_interpolated[not_nan], P_vals[not_nan])

print(coefficient)

PearsonRResult(statistic=0.5117409518260079, pvalue=3.9673140669436364e-33)
